In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import efficientnet.tfkeras as efn #Convolutional Neural Network architecture
from efficientnet.keras import center_crop_and_resize, preprocess_input
from sklearn.utils import class_weight

## Engineering Features

In [3]:
tf.device('')

[]

In [2]:
rootFolder = "bird_data"
wavFolder = os.path.join("bird_data","wav_files")
wavFiles = os.listdir(wavFolder)
bird_calls_df = pd.read_csv(os.path.join(rootFolder, "bird_calls.csv"))
bird_calls_df.head()

,fname,label
0,American_Goldfinch_XC114342.wav,American_Goldfinch
1,American_Goldfinch_XC124312.wav,American_Goldfinch
2,American_Goldfinch_XC133564.wav,American_Goldfinch
3,American_Goldfinch_XC133565.wav,American_Goldfinch
4,American_Goldfinch_XC141469.wav,American_Goldfinch


In [3]:
mel_features_df = pd.read_csv(os.path.join(rootFolder, "mel_features.csv"))
mfcc_features_df = pd.read_csv(os.path.join(rootFolder, "mfcc_features.csv"))

In [4]:
mel_features_df.head()

,label,feature
0,American_Goldfinch,[[-80. -80. -80. ... -80. -80. -80.]\n [-80. -...
1,American_Goldfinch,[[-80. -80. -80. ... -80. -80. -80.]\n [-80. -...
2,American_Goldfinch,[[-80. -80. -80. ... -...
3,American_Goldfinch,[[-80. -80. -80. ... -...
4,American_Goldfinch,[[-80. -80. -80. ... -80. -80. -80.]\n [-80. -...


In [5]:
mfcc_features_df.head()

,label,feature
0,American_Goldfinch,[[ -2.48275794 -3.56181979 -2.70022824 ... ...
1,American_Goldfinch,[[ 0. -1.13543094 -0.34263572 ... ...
2,American_Goldfinch,[[ -1.06088357 -5.78667663 -4.09501159 ... ...
3,American_Goldfinch,[[ -0.7841282 -8.00023103 -6.01809665 ... ...
4,American_Goldfinch,[[ -1.33764821 -1.20825628 -0.57548347 ... ...


In [9]:
# labels
bird_classes = list(np.unique(bird_calls_df.label))
bird_classes

['American_Goldfinch',
 'American_Robin',
 'Barn_Swallow',
 'Blue-grey_Gnatcatcher',
 'Blue_Jay',
 'Carolina_Chickadee',
 'Carolina_Wren',
 'Cedar_Waxwing',
 'Northern_Cardinal',
 'Ruby-crowned_Kinglet']

# Convolutional Neural Network

In [10]:
# Batch and target size
size_batch = 32
size_target = (288,288)
